In [1]:
import sys
import bluetooth
import obd
import can

In [ ]:
"""PyBluez simple example inquiry.py
Performs a simple device inquiry followed by a remote name request of each
discovered device
Author: Albert Huang <albert@csail.mit.edu>
$Id: inquiry.py 401 2006-05-05 19:07:48Z albert $
"""

print("Performing inquiry...")

nearby_devices = bluetooth.discover_devices(duration=8, lookup_names=True,
                                            flush_cache=True, lookup_class=False)

print(f"Found {len(nearby_devices)} devices")

for addr, name in nearby_devices:
    try:
        print(f"   {addr} - {name}")
    except UnicodeEncodeError:
        print(f"   {addr} - {name.encode('utf-8', 'replace')}")

In [ ]:
"""PyBluez simple example sdp-browse.py
Displays services being advertised on a specified bluetooth device.
Author: Albert Huang <albert@csail.mit.edu>
$Id: sdp-browse.py 393 2006-02-24 20:30:15Z albert $
"""

# if len(sys.argv) < 2:
#     print("Usage: sdp-browse.py <addr>")
#     print("   addr - can be a bluetooth address, \"localhost\", or \"all\"")
#     sys.exit(2)

# target = sys.argv[1]
# if target == "all":
#     target = None

target = "30:AE:A4:1C:C5:3E" # myESP32 bluetooth address
# target = "84:CF:BF:8D:C3:F8" # Kai_FP3 bluetooth address

services = bluetooth.find_service(address=target)

if len(services) > 0:
    print(f"Found {len(services)} services on {target}.")
else:
    print("No services found.")

for svc in services:
    print("\nService Name:", svc["name"])
    print("    Host:       ", svc["host"])
    print("    Description:", svc["description"])
    print("    Provided By:", svc["provider"])
    print("    Protocol:   ", svc["protocol"])
    print("    channel/PSM:", svc["port"])
    print("    svc classes:", svc["service-classes"])
    print("    profiles:   ", svc["profiles"])
    print("    service id: ", svc["service-id"])

In [ ]:
"""PyBluez simple example rfcomm-client.py
Simple demonstration of a client application that uses RFCOMM sockets intended
for use with rfcomm-server.
Author: Albert Huang <albert@csail.mit.edu>
$Id: rfcomm-client.py 424 2006-08-24 03:35:54Z albert $
"""



addr = None

# if len(sys.argv) < 2:
#     print("No device specified. Searching all nearby bluetooth devices for "
#           "the SampleServer service...")
# else:
#     addr = sys.argv[1]
#     print("Searching for SampleServer on {}...".format(addr))

# search for the SPP service
addr = "30:AE:A4:1C:C5:3E" # myESP32 bluetooth address
service_matches = bluetooth.find_service(name="SPP_SERVER", address=addr)

# # search for the SPP service
# addr = "84:CF:BF:8D:C3:F8" # Kai_FP3 bluetooth address
# service_matches = bluetooth.find_service(name="OBEX Object Push", address=addr)

if len(service_matches) == 0:
    print("Couldn't find the SPP service.")

else:
    first_match = service_matches[0]
    port = first_match["port"]
    name = first_match["name"]
    host = first_match["host"]

    print(f"Connecting to \"{name}\" on {host} channel {port}")

    # Create the client socket
    sock = bluetooth.BluetoothSocket(bluetooth.RFCOMM)
    sock.connect((host, port))

    print("Connected. Type something...")
    while True:
        data = input()
        if not data:
            break
        sock.send(data)

    sock.close()
    print("Connection closed")

In [ ]:
    port = 7
    name = "SPP_SERVER"
    host = "30:AE:A4:1C:C5:3E"

    print(f"Connecting to \"{name}\" on {host} channel {port}")

    # Create the client socket
    sock = bluetooth.BluetoothSocket(bluetooth.RFCOMM)

    try:
        sock.connect((host, port))

        try:
            print("Connected. Type something...")
            while True:
                data = input()
                if not data:
                    break
                sock.send(data)
                msg = sock.recv(1024)
                print(msg)
        except Exception as err:
            print(err)
            raise
        finally:
            sock.close()
            print("Connection closed")
            
    except Exception as err:
        print(err)
        raise

In [61]:
    port = 7
    name = "SPP_SERVER"
    host = "30:AE:A4:1C:C5:3E"

    print(f"Connecting to \"{name}\" on {host} channel {port}")

    # Create the client socket
    sock = bluetooth.BluetoothSocket(bluetooth.RFCOMM)

    try:
        sock.connect((host, port))

        try:
            cmd = "N" # Serial number
            # cmd = "V" # Version
            # cmd = "S6" # 500 kbit/s
            # cmd = "F" # Status flags
            # cmd = "W1" # Singel mode filter
            # cmd = "Z1" # timestamps
            # cmd = "X1" # auto poll
            # cmd = "C" # Close
            # cmd = "O" # Open
            # cmd = "Q1" # auto startup
            # cmd = "P" # poll
            # cmd = "A" # poll all
            # cmd = "t123400112233"
            cmd = bytes(cmd + "\r", encoding='utf8')
            print(f"Connected. Sending cmd {cmd} ...")
            sock.send(cmd)
            msg = sock.recv(1024)
            print(msg)
        except Exception as err:
            print(err)
            raise
        finally:
            sock.close()
            print("Connection closed")
            
    except Exception as err:
        print(err)
        raise

Connecting to "SPP_SERVER" on 30:AE:A4:1C:C5:3E channel 7
Connected. Sending cmd b'N\r' ...
b'N1118\r'
Connection closed


In [6]:
obd.commands.SPEED

OBDCommand('SPEED', 'Vehicle Speed', b'010D', 3, raw_string, ecu=ECU.ENGINE, fast=True)

In [2]:
can_bt_bus = can.interface.Bus(bustype="slcan", channel="/dev/rfcomm0", ttyBaudrate=115200, bitrate=500000)

In [3]:
can_bt_bus.shutdown()